In [52]:
#Import libraries
import numpy as np
import pandas as pd

#For plotting
import seaborn as sns
import matplotlib.pyplot as plt

# set warnings to ignore
import warnings
warnings.filterwarnings('ignore')

#tqdm
from tqdm import tqdm
tqdm.pandas()

#Mongodb
import pdmongo as pdm
from helpers import MongoDAO


# Connect to the database


In [53]:
df_sub = pd.read_csv('../datasets/Dataset do Case de Analista_Cientista de Dados/subscription.csv')


In [54]:
df_sub['account_id'] = df_sub['account_id'].astype(str)

In [55]:
df_sub['start_date'] = pd.to_datetime(df_sub.start_date)
df_sub['end_date'] = pd.to_datetime(df_sub.end_date)

In [56]:
df_sub['month_year'] = df_sub.start_date.dt.to_period('M').apply(str)


In [57]:
df_sub.head()

,id,account_id,product,start_date,end_date,mrr,month_year
0,0,1,standard,2020-01-06,2020-02-06,9.99,2020-01
1,1,1,standard,2020-02-06,2020-03-06,9.99,2020-02
2,2,1,standard,2020-03-06,2020-04-06,9.99,2020-03
3,3,1,standard,2020-04-06,2020-05-06,9.99,2020-04
4,4,1,standard,2020-05-06,2020-06-06,9.99,2020-05


In [58]:
month = '2020-05'
accounts = df_sub.account_id.unique().tolist()
arr_churn_accounts = []

for account in tqdm(accounts):
    df_sub_account = df_sub[df_sub.account_id == account]
    if len(df_sub_account[df_sub_account.month_year == month]) == 0:
        arr_churn_accounts.append(account)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14641/14641 [00:50<00:00, 291.05it/s]


In [59]:
arr_churn_accounts[:5]

['14', '15', '16', '26', '29']

In [60]:
df_sub[df_sub.account_id=='14']

,id,account_id,product,start_date,end_date,mrr,month_year
65,66,14,standard,2020-01-21,2020-02-21,9.99,2020-01
66,67,14,standard,2020-02-21,2020-03-21,9.99,2020-02
67,68,14,standard,2020-03-21,2020-04-21,9.99,2020-03
68,69,14,standard,2020-04-21,2020-05-21,9.99,2020-04


### Creating feature store

In [61]:
df = pd.DataFrame(columns=['account_id'])

In [62]:
df['account_id'] = df_sub.account_id.unique()

In [63]:
df['churn'] = df.account_id.apply(lambda x: 1 if x in arr_churn_accounts else 0)

In [64]:
df.churn.value_counts()

0    12386
1     2255
Name: churn, dtype: int64

In [65]:
def get_ltv(account_id, df_sub):
    '''
    Returns the lifetime value of an account and number of months
    '''
    df = df_sub.copy()
    df = df[df.account_id == account_id]
    sum = df['mrr'].sum()
    months = df.shape[0]
    return sum, months


In [66]:
df['ltv'], df['months'] = zip(*df.account_id.apply(lambda x: get_ltv(x, df_sub)))

In [67]:
df.head(10)

,account_id,churn,ltv,months
0,1,0,49.95,5
1,2,0,49.95,5
2,3,0,49.95,5
3,4,0,49.95,5
4,5,0,49.95,5
5,6,0,49.95,5
6,7,0,49.95,5
7,8,0,49.95,5
8,9,0,49.95,5
9,10,0,49.95,5


In [68]:
pd.DataFrame({
    'columns' : df_sub.columns,
    'dtypes' : df_sub.dtypes,
    'nulls' : df_sub.isnull().sum(),
    'nulls_percentage' : df_sub.isnull().sum()/df_sub.shape[0]*100,
    'unique' : df_sub.nunique()
})

,columns,dtypes,nulls,nulls_percentage,unique
id,id,int64,0,0.0,55150
account_id,account_id,object,0,0.0,14641
product,product,object,0,0.0,1
start_date,start_date,datetime64[ns],0,0.0,152
end_date,end_date,datetime64[ns],0,0.0,148
mrr,mrr,float64,0,0.0,1
month_year,month_year,object,0,0.0,5


In [69]:
#Inserindo a coluna product _ cardinaldiade 1
df['product'] = df_sub['product'].unique()[0]

### Dados da tabela account

In [70]:
df_account = pd.read_csv('../datasets/Dataset do Case de Analista_Cientista de Dados/account.csv')

In [71]:
df_account.head()


,id,channel,date_of_birth,country
0,1,appstore2,1948-09-10,CN
1,2,appstore1,1952-11-06,AU
2,3,appstore1,2002-11-10,GB
3,4,appstore1,1976-06-18,US
4,5,appstore1,1975-11-18,KR


In [72]:
pd.DataFrame(
    {
        'columns' : df_account.columns,
        'dtypes' : df_account.dtypes,
        'nulls' : df_account.isnull().sum(),
        'nulls_percentage' : df_account.isnull().sum()/df_account.shape[0]*100,
        'unique' : df_account.nunique()
    }
)

,columns,dtypes,nulls,nulls_percentage,unique
id,id,int64,0,0.000000,14641
channel,channel,object,0,0.000000,3
date_of_birth,date_of_birth,object,0,0.000000,10995
country,country,object,1611,11.003347,26


In [73]:
df_account['account_id'] = df_account.id.apply(str)

In [74]:
df = pd.merge(df, df_account, on='account_id')

In [75]:
df.drop(['id'], axis=1, inplace=True)

### Dados da tabela evento

In [76]:
df_event = pd.read_csv('../datasets/Dataset do Case de Analista_Cientista de Dados/event.csv')

In [77]:
df_event_type = pd.read_csv('../datasets/Dataset do Case de Analista_Cientista de Dados/event_type.csv')

In [78]:
df_event = pd.merge(df_event, df_event_type, on='event_type_id')

In [79]:
df_event.head()
df_event['account_id'] = df_event.account_id.apply(str)
df_event['event_type_id'] = df_event.event_type_id.apply(str)


In [80]:
df_event.head()

,account_id,event_time,event_type_id,event_type_name
0,1,2020-01-06 03:20:00,0,post
1,1,2020-01-06 00:55:29,0,post
2,1,2020-01-07 04:08:23,0,post
3,1,2020-01-07 08:45:06,0,post
4,1,2020-01-07 05:50:52,0,post


In [81]:
df_event.event_type_name.unique()

array(['post', 'newfriend', 'like', 'adview', 'message', 'reply',
       'dislike', 'unfriend'], dtype=object)

In [82]:
def numero_de_eventos_por_tipo(account_id, df_event):
    '''
    Returns the number of events of an account by type
    '''
    df = df_event.copy()
    df = df[df.account_id == account_id]
    df = pd.DataFrame(df.event_type_name.value_counts()).reset_index().T
    df.columns = df.iloc[0]
    df.drop('index',axis=0,inplace=True)
    df = df.reset_index()
    df['account_id'] = account_id
    return df

In [83]:
arr_df = []
accounts = df.account_id.unique().tolist()

for account in tqdm(accounts):
    df_event_account = df_event[df_event.account_id == account]
    df_event_account = numero_de_eventos_por_tipo(account, df_event_account)
    arr_df.append(df_event_account)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14641/14641 [2:52:47<00:00,  1.41it/s]


In [84]:
_df = pd.concat(arr_df).drop('index',axis=1)
df = pd.merge(df, _df, on='account_id', how='left')

### Write dataframe to MongoDB

In [91]:
from helpers import MongoDAO

In [92]:
mongo = MongoDAO('admin','admin','localhost',27017)

In [93]:
mongo.set_db('bra_paralelo')

In [94]:
mongo.set_collection('feature_store')

In [95]:
account_store_collection = mongo.get_collection()

In [96]:
account_store_collection.insert_many(df.to_dict('records'))